# CNN Hyperparameters tuning using Hyperas and Hyperopt

In [1]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import cv2
from random import shuffle
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.utils import np_utils
import seaborn as sns
from keras.initializers import RandomNormal
import time
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.preprocessing import image
from keras.callbacks import EarlyStopping
import keras
%matplotlib inline

Using TensorFlow backend.


In [2]:
# keras.__version__

In [9]:
def data():
    train_data = 'D:/Office/MS/finding_sign_on_form/keras_model/train_data/'
    test_data = 'D:/Office/MS/finding_sign_on_form/keras_model/test_data/'    
    def one_hot_label(img):
        label = img[0]
        if label == 'h':
            ohl = np.array([1,0])
        elif label == 'l' or 'p':
            ohl = np.array([0,1])
        return ohl

    def train_data_with_label():
        train_images = []
        train_data_imgs = [file for file in os.listdir(train_data) if file.endswith(".jpg") or file.endswith(".JPG")]
        for i in train_data_imgs:
            path = os.path.join(train_data, i)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 32))
            img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)[1]
            train_images.append([np.array(img), one_hot_label(i)])
        shuffle(train_images)
        return train_images

    def test_data_with_label():
        test_images = []
        test_data_imgs = [file for file in os.listdir(test_data) if file.endswith(".jpg") or file.endswith(".JPG")]
        for i in test_data_imgs:
            path = os.path.join(test_data, i)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 32))
            img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)[1]
            test_images.append([np.array(img), one_hot_label(i)])
        shuffle(test_images)
        return test_images
    training_images = train_data_with_label()
    testing_images = test_data_with_label()

    tr_img_data = np.array([i[0] for i in training_images]).reshape(-1,100,32,1)
    tr_lbl_data = np.array([i[1] for i in training_images])

    tst_img_data = np.array([i[0] for i in testing_images]).reshape(-1,100,32,1)
    tst_lbl_data = np.array([i[1] for i in testing_images])
    X_train = tr_img_data.astype('float32')
    y_train = tr_lbl_data
    X_val = tst_img_data.astype('float32')
    y_val = tst_lbl_data
    X_train /= 255
    X_val /= 255
    return X_train, y_train, X_val, y_val

In [12]:
def model(X_train, y_train, X_val, y_val):
    model = Sequential()
    model_choice = {{choice(['one', 'two', 'three', 'four'])}}
    if model_choice == 'one':
        model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='valid', activation='relu', input_shape=(100,32,1)))
        model.add(MaxPool2D(pool_size=3, strides=2))
        
        model.add(Conv2D(filters=50, kernel_size=3, strides=1, padding='valid', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
        model.add(Dropout({{uniform(0, 1)}}))

        model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='valid', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
        
    elif model_choice == 'two':
        model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(100,32,1)))
        model.add(MaxPool2D(pool_size=3, strides=2))
        
        model.add(Conv2D(filters=50, kernel_size=3, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
        model.add(Dropout({{uniform(0, 1)}}))

        model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
   
    elif model_choice == 'three':
        model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='valid', activation='relu', input_shape=(100,32,1)))
        model.add(MaxPool2D(pool_size=3, strides=2))

        model.add(Conv2D(filters=50, kernel_size=3, strides=1, padding='valid', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
        model.add(Dropout({{uniform(0, 1)}}))

        model.add(Conv2D(filters=80, kernel_size=3, strides=1, padding='valid', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
    elif model_choice == 'four':
        model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(100,32,1)))
        model.add(MaxPool2D(pool_size=3, strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        
        model.add(Conv2D(filters=50, kernel_size=3, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))
        model.add(Dropout({{uniform(0, 1)}}))

        model.add(Conv2D(filters=80, kernel_size=3, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=2, strides=2))

    model.add(Flatten())
    model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(2, activation='softmax'))
    
    adam = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=adam)
    model.fit(X_train, y_train,
              batch_size=64,
              nb_epoch=15,
              verbose=2,
              validation_data=(X_val, y_val))
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [13]:
X_train,y_train,X_val,y_val = data()
# print(len(X_train), len(Y_train), len(X_val), len(Y_val))
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name='CNN_hyperparameters')

>>> Imports:
#coding=utf-8

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D
except:
    pass

try:
    from keras.layers import Activation, Dropout, Flatten, Dense
except:
    pass

try:
    import cv2
except:
    pass

try:
    from random import shuffle
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from PIL import Image
except:
    pass

try:
    import os
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import *
except:
    pass

try:
    from keras.optimizers import *
except:
    pass

try:
    

  0%|                                                                             | 0/30 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From C:\Users\Shubham\.conda\envs\shub\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\Shubham\.conda\envs\shub\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '



Instructions for updating:
Use tf.cast instead.
Train on 7783 samples, validate on 2000 samples                                                                        
Epoch 1/15                                                                                                             
 - 8s - loss: 0.3670 - acc: 0.8746 - val_loss: 0.1561 - val_acc: 0.9460                                                

Epoch 2/15                                                                                                             
 - 3s - loss: 0.1841 - acc: 0.9347 - val_loss: 0.5253 - val_acc: 0.8225                                                

Epoch 3/15                                                                                                             
 - 3s - loss: 0.1409 - acc: 0.9516 - val_loss: 4.3585 - val_acc: 0.5640                                                

Epoch 4/15                                                                                                   

C:\Users\Shubham\.conda\envs\shub\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '



...

Train on 7783 samples, validate on 2000 samples                                                                        
Epoch 1/15                                                                                                             
 - 9s - loss: 0.2881 - acc: 0.9098 - val_loss: 0.1584 - val_acc: 0.9455                                                

Epoch 2/15                                                                                                             
 - 3s - loss: 0.1394 - acc: 0.9517 - val_loss: 0.0668 - val_acc: 0.9775                                                

Epoch 3/15                                                                                                             
 - 3s - loss: 0.1049 - acc: 0.9607 - val_loss: 0.4621 - val_acc: 0.8320                                                

Epoch 4/15                                                                                                             
 - 3s - loss: 0.0920 - acc: 0.9663 - 

C:\Users\Shubham\.conda\envs\shub\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '



Train on 7783 samples, validate on 2000 samples                                                                        
Epoch 1/15                                                                                                             
 - 9s - loss: 0.4491 - acc: 0.8458 - val_loss: 0.0677 - val_acc: 0.9810                                                

Epoch 2/15                                                                                                             
 - 3s - loss: 0.2134 - acc: 0.9228 - val_loss: 0.0726 - val_acc: 0.9805                                                

Epoch 3/15                                                                                                             
 - 3s - loss: 0.1575 - acc: 0.9459 - val_loss: 0.0536 - val_acc: 0.9800                                                

Epoch 4/15                                                                                                             
 - 3s - loss: 0.1259 - acc: 0.9537 - 

C:\Users\Shubham\.conda\envs\shub\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '



Train on 7783 samples, validate on 2000 samples                                                                        
Epoch 1/15                                                                                                             
 - 9s - loss: 4.2374 - acc: 0.5079 - val_loss: 0.6972 - val_acc: 0.4910                                                

Epoch 2/15                                                                                                             
 - 3s - loss: 2.8914 - acc: 0.5227 - val_loss: 0.5989 - val_acc: 0.7565                                                

Epoch 3/15                                                                                                             
 - 3s - loss: 1.3248 - acc: 0.5515 - val_loss: 0.5165 - val_acc: 0.8890                                                

Epoch 4/15                                                                                                             
 - 3s - loss: 0.7112 - acc: 0.6109 - 

In [14]:
print(best_run)

{'Dense': 2, 'Dense_1': 2, 'Dropout': 0.5702318469279729, 'Dropout_1': 0.6264617360166798, 'Dropout_2': 0.1904090881738011, 'Dropout_3': 0.08063845654367285, 'Dropout_4': 0.9772817488940724, 'Dropout_5': 0.6883693507416478, 'Dropout_6': 0.527899792364496, 'choiceval': 1, 'model_choice': 2}


In [15]:
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 96, 28, 32)        832       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 47, 13, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 45, 11, 50)        14450     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 22, 5, 50)         0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 22, 5, 50)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 20, 3, 80)         36080     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 10, 1, 80)         0         
__________